In [4]:
import torch

In [32]:
x = torch.ones(3, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

out.backward(retain_graph=True)     # keep graph alive for another backward
print(x.grad)                       # tensor([6., 6., 6.])
#x.detach()
x.grad = None                       # clear accumulation
y.backward(torch.tensor([1., 0.5, 0.1]))
print(x.grad)                       # tensor([1.0000, 0.5000, 0.1000])


tensor([6., 6., 6.])
tensor([1.0000, 0.5000, 0.1000])


In [24]:
x.grad

tensor([7.0000, 6.5000, 6.1000])

In [ ]:
#coding practise 
# 1. manual update gradient descent 
x = torch.linspace(-3.14, 3.14, 2000)
y = torch.sin(x)
lr = 0.01
a = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)
c = torch.tensor(1.0, requires_grad=True)
d = torch.tensor(1.0, requires_grad=True)
epochs = 10 
for _ in range(epochs):
    y_pred = a + b*x + c*x**2 + d*x**3
    loss = (y_pred-y)**2  # shape of both is nx 1 
    loss = loss.mean()
    print(loss.item())
    loss.backward() 
    with torch.no_grad():
        a -= lr*a.grad 
        b -= lr*b.grad 
        c -= lr*c.grad 
        d -= lr*d.grad 
        a.grad = b.grad = c.grad = d.grad = None
    print(a,b,c,d)
#Single check for you: why is set_to_none=True often better than grad.zero_() in training loops? 
# Give me the two practical advantages in one line each.


197.4093475341797
tensor(0.9142, requires_grad=True) tensor(0.5646, requires_grad=True) tensor(0.5446, requires_grad=True) tensor(-2.0589, requires_grad=True)
563.1483764648438
tensor(0.8601, requires_grad=True) tensor(1.3496, requires_grad=True) tensor(0.2722, requires_grad=True) tensor(3.4538, requires_grad=True)
1800.62890625
tensor(0.8250, requires_grad=True) tensor(-0.0648, requires_grad=True) tensor(0.1096, requires_grad=True) tensor(-6.4812, requires_grad=True)
5837.61279296875
tensor(0.8013, requires_grad=True) tensor(2.4847, requires_grad=True) tensor(0.0126, requires_grad=True) tensor(11.4234, requires_grad=True)
18955.60546875
tensor(0.7844, requires_grad=True) tensor(-2.1097, requires_grad=True) tensor(-0.0450, requires_grad=True) tensor(-20.8441, requires_grad=True)
61563.44140625
tensor(0.7717, requires_grad=True) tensor(6.1705, requires_grad=True) tensor(-0.0791, requires_grad=True) tensor(37.3079, requires_grad=True)
199949.015625
tensor(0.7615, requires_grad=True) tens

In [55]:
# 2. Problem 2 — Custom autograd op
#Implement a custom Function for 𝑦=𝑥3. Use ctx.save_for_backward and write the correct backward.
import torch 
from torch.autograd import Function 

class CustomAutograd(Function):
    def forward(ctx, input ):
        # save the input 
        ctx.save_for_backward(input)
        return input**3
    def backward(ctx, grad_output):
        x, = ctx.saved_tensors
        return grad_output* 3*(x**2)
    

x = torch.tensor([1.0], requires_grad=True)
y = CustomAutograd.apply(x)         # <-- use .apply
y.backward(torch.ones_like(y))


In [70]:
# 3. Linear Layer from scratch 
import torch.nn as nn
import math
import torch
import torch.nn as nn
import torch.nn.init as init

class MyLinear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.weight = nn.Parameter(torch.empty(out_features, in_features))
        if bias:
            self.bias = nn.Parameter(torch.empty(out_features))
        else:
            self.register_parameter('bias', None)
        
    def forward(self, input):
        y = input @ self.weight.T
        if self.bias: 
            out = out + self.bias
        return y 
x = torch.Tensor(1, 10 )

layer = MyLinear(10, 1)
layer(x)

tensor([[0.]], grad_fn=<MmBackward0>)

In [71]:
#pin memory 
import torch, time

# Make sure CUDA is available
device = torch.device("cuda")

# A big fake batch
x_cpu = torch.randn(1024, 1024, 256)  # ~1GB tensor

# Copy without pinned memory
start = time.time()
for _ in range(50):
    y = x_cpu.to(device, non_blocking=False)
torch.cuda.synchronize()
no_pin_time = time.time() - start

# Copy WITH pinned memory
x_pinned = x_cpu.pin_memory()
start = time.time()
for _ in range(50):
    y = x_pinned.to(device, non_blocking=True)
torch.cuda.synchronize()
pin_time = time.time() - start

print(f"Without pin_memory: {no_pin_time:.3f}s")
print(f"With pin_memory:    {pin_time:.3f}s")


AssertionError: Torch not compiled with CUDA enabled

In [85]:
#Problem 4 — forward propogation 
import numpy as np
# torch.Tensor is a fatory funciton to make data torch.Tensor(2,3) makes a tensor of shape 2,3 
# whereas torch.tensor makes a tensor of [2, 3] converts this to a tensor 
X = np.array([[2, 3]])
W = np.array([[1], [0.5]])
b = np.array([1])
import torch 
X = torch.tensor(X, dtype = torch.float32)
W = torch.tensor(W, dtype = torch.float32)
b = torch.tensor(b, dtype = torch.float32)
y = torch.matmul(X, W) + b 
y = torch.sigmoid(y)
print(y)
#print(y.detach().numpy())

tensor([[0.9890]])


In [ ]:
# gini imputiry for a decisin tree 
import torch
labels = torch.tensor([0, 0, 0, 0])
n = labels.numel()
print(n)
max_class = int(labels.max().item()) 
counts = torch.bincount(labels.to(torch.int64), minlength=max_class + 1)
print(counts)
probs = counts.float() / n
gini = 1.0 - torch.sum(probs * probs).item()

4
tensor([4])


In [101]:
input_vector = [1.0, 2.0]

input_vector = torch.tensor(input_vector, dtype = torch.float32)
W = torch.tensor([1,-1], dtype = torch.float32) # 1, 2 
b = torch.tensor(0.5,dtype = torch.float32) # 1 
relu = torch.nn.Sigmoid()
z = torch.matmul(input_vector, W) + b 
relu(z) 

tensor(0.3775)

In [121]:
# question 5 cmpute the attention scores in torch 
import torch
import math
import warnings
warnings.filterwarnings("ignore")
torch.manual_seed(0)
query = torch.tensor([[1.0, 0.0]])
key = torch.tensor([[1.0, 0.0], [0.0, 1.0]])
value = torch.tensor([[2.0], [3.0]])
dk= key.shape[-1]
print(dk )
print(query.shape)
print(key.shape)
dot = torch.matmul(query, key.T)
print(dot.shape)
dot = dot/(dk**0.5) 
softmax  = torch.nn.Softmax()
attention_scores = softmax(dot)
print(attention_scores)
attention = torch.matmul(attention_scores, value)
print(attention.shape)

2
torch.Size([1, 2])
torch.Size([2, 2])
torch.Size([1, 2])
tensor([[0.6698, 0.3302]])
torch.Size([1, 1])


In [ ]:
import numpy as np
np.random.seed(42)
data = np.array([[1, 2], [1, 4], [5, 2]], dtype=float)
k = 2
X = torch.tensor(data, dtype=torch.float32)   # n d  3 x 2 
seed = 4
max_iter = 10 
#g = torch.Generator().manual_seed(seed)
centroids = X[:k].clone() # kx d 
for iteration in range(max_iter):
    # calculate distrance to k points for each of these n points 
    x2 =  (X**2).sum(dim=1, keepdim=True) 
    #print(x2.shape) # 3 1 
    c2 = (centroids**2).sum(dim=1,keepdim = True) # (1, k)
    #print(c2.shape)
    product = torch.matmul(X,centroids.T) # nxd dxk

    distances = x2 + c2.T - 2*product # nx k 
    # we have the distances of each x poitn from the c centroids, now we choose 
    labels = torch.argmin(distances, dim=1)
    print(distances.shape) # nx c 
    # for each n chooose the minimum centroid 
    new_centroids = centroids.clone()
    for i in range(k):
        # for each centroid assign n to it if labels[i]==i 
        points_in_cluster = X[labels == i]
        if len(points_in_cluster) > 0:
            # Calculate the mean and update the centroid
            new_centroids[i] = points_in_cluster.mean(dim=0)
    if torch.allclose(centroids, new_centroids):
        print(f"Converged after {iteration + 1} iterations.")
        break
    centroids = new_centroids


torch.Size([3, 2])
torch.Size([3, 2])
Converged after 2 iterations.


In [ ]:
# torch coding 